In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta

In [2]:
#assign dates to dataframe index
PFE_df = pd.read_csv("/Users/fahad/project_repo/data/final/PFE/PFE_normalised.csv", index_col=0)
dates = PFE_df.index
PFE_df['Date'] =dates


In [3]:
PFE_df['Date'] = pd.to_datetime(PFE_df.Date, format ="%Y-%m-%d")

In [4]:
PFE_2010 = pd.read_csv("/Users/fahad/project_repo/data/final/PFE/PFE_10yr.csv", index_col=0)

In [9]:
#ADD LABELS TO THE TRAINING AND TEST DATAFRAMES AND RETURN DICTIONARIES WITH DATE:LABEL FORMAT
PFE_2010_labels = labelling(PFE_2010)

In [10]:
PFE_2010_flattened= get_ndays_monthly_flattened(PFE_2010,PFE_df,30)


In [11]:
PFE_2010_labels_df = pd.Series(PFE_2010_labels).to_frame("Label")

In [12]:
#CREATE DATAFRAME FROM FLATTENED DICTIONARY OF ARRAYS FOR TRAINING AND TEST DATA
PFE_2010_flat_df = pd.DataFrame.from_dict(PFE_2010_flattened).T


In [13]:
#JOIN THE LABEL DATAFRAME AND THE FLATTENED DATAFRAME ON THE INDEX WHICH IS THE DATE
PFE_2010_flat_df = PFE_2010_flat_df.merge(PFE_2010_labels_df, left_index=True, right_index=True)


In [14]:
PFE_2010_flat_df

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,Label
2019-07-31,0.617228,0.615156,0.621200,0.604713,0.115180,1.000000,0.466739,0.493114,0.498150,0.230835,...,0.876808,0.021522,0.728993,0.463320,0.665142,0.688396,0.316393,0.0,0.0,1
2019-06-28,0.748196,0.757012,0.742249,0.726567,0.304842,0.926953,0.568587,0.493114,0.498150,0.376194,...,0.841189,0.063090,0.382044,0.336176,0.776663,0.779627,0.267378,0.0,0.0,1
2019-05-31,0.833539,0.846548,0.863573,0.850830,0.060310,0.509692,0.373642,0.679894,0.724765,0.291868,...,0.802892,0.071961,0.335496,0.423660,0.876191,0.800077,0.167756,0.0,0.0,0
2019-04-30,0.797648,0.803128,0.824232,0.809855,0.044889,0.311784,0.322615,0.837808,0.800077,0.191453,...,0.755758,0.128012,0.431623,0.429342,0.874169,0.857251,0.282347,0.0,0.0,0
2019-03-29,0.751938,0.751079,0.769485,0.747991,0.064402,0.171534,0.422506,0.893472,0.812560,0.245639,...,0.807445,0.050360,0.371824,0.398490,0.819811,0.864540,0.210617,0.0,0.0,1
2019-02-28,0.811013,0.814455,0.837437,0.820300,0.043467,0.473608,0.523455,0.837882,0.891000,0.213198,...,0.811730,0.053247,0.341811,0.407936,0.849167,0.848975,0.243194,0.0,0.0,1
2019-01-31,0.832398,0.837379,0.858070,0.837440,0.042007,0.569541,0.568811,0.843709,0.849283,0.278347,...,0.832084,0.073144,0.582221,0.561365,0.871202,0.856835,0.350814,0.0,0.0,0
2018-12-31,0.773323,0.797735,0.800847,0.801285,0.106975,0.345351,0.447437,0.847986,0.848975,0.295929,...,0.842260,0.115543,0.563185,0.448373,0.844004,0.873016,0.492999,0.0,0.0,1
2018-11-30,0.833467,0.846009,0.858070,0.838243,0.071870,0.484378,0.512936,0.861336,0.886423,0.344457,...,0.836904,0.081545,0.320663,0.301267,0.855067,0.873016,0.447721,0.0,0.0,1
2018-10-31,0.879979,0.907767,0.910066,0.906535,0.073807,0.608270,0.638177,0.864582,0.873016,0.475680,...,0.866095,0.061979,0.509051,0.546346,0.920637,0.914317,0.453039,0.0,0.0,0


In [15]:
export = PFE_2010_flat_df.to_csv("/Users/fahad/project_repo/data/final/PFE/PFE_2010_pre_signals.csv")

In [7]:
def labelling(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    empty_dict = dict()
    prev_close =0
    counter = 0
    for index,cols in dataframe.iterrows():
        counter +=1
        current_close = cols['Close']
        if float(prev_close) <= float(current_close):
            #signals an increase in price between last month and next month
            empty_dict[index] = 1
        elif float(prev_close) >= float(current_close):
            empty_dict[index]=0

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [8]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            if (len(temp_df.values.flatten())==360):
                empty_dict[month_key]= temp_df.values.flatten()
                count +=1      
            else:
                pass

        else:
            pass
       
    return empty_dict
    